# Data Sources to be Used for The Project

## Data Sources

For solving this problem, data from two sources will be leveraged-

1. Location data titled ["California Counties"][1] provided in [**California Open Data Portal**][2] provided by **Government of California** for the geographical location data. This data is in `.csv` format.
2. The **[Foursquare][3] API** for information about established restaurants and other relevant detailed information about the same.
3. County-wise population data from **[US Government Census site][4]**.([File Link][5]). This file is in `.xlsx` format. Only the latest data (year 2019) has been kept, and it has be turned into a CSV file for further cleaning.
4. County-wise Real GDP data provided by **[Bureau of Economic Analysis, U.S. Department of Commerce][6]**. ([File Link][7]). This data is also in `.xlsx` format. Irrelevant data has been truncated and the file has been converted to CSV format for further cleaning.


\[3\] Citation: *Annual Estimates of the Resident Population for Counties in California: April 1, 2010 to July 1, 2019 (CO-EST2019-ANNRES-06) Source: U.S. Census Bureau, Population Division Release Date: March 2020*



With the data collected from these sources, analysis will be carried out for solving the business problem.



[1]: https://data.ca.gov/dataset/california-counties/resource/8302d20a-3bab-4687-9a32-b2c84b6ee603
[2]: https://data.ca.gov/
[3]: https://foursquare.com/
[4]: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html
[5]: https://www2.census.gov/programs-surveys/popest/tables/2010-2019/counties/totals/co-est2019-annres-06.xlsx
[6]: https://www.bea.gov/
[7]: https://www.bea.gov/system/files/2019-12/lagdp1219.xlsx

## Example of data

In this section, I will show you some examples of the form of the data that will be used for analysis.

In [1]:
# importing essential libraries
import pandas as pd

import folium

import requests

#### 1. Location Data

Location data is downloaded from above mentioned source.

In [2]:
# downloading the data in csv format
! wget -q -O 'ca_counties_location.csv' https://data.edd.ca.gov/api/views/bpwh-bcb3/rows.csv?accessType=DOWNLOAD

ca_counties_df = pd.read_csv('ca_counties_location.csv')
ca_counties_df.head()

,Latitude,the_geom,Name,Longitude
0,41.743438,MULTIPOLYGON (((-124.21284783991833 41.8708173...,Del Norte,-123.896950
1,41.592777,MULTIPOLYGON (((-123.51814169611895 42.0011639...,Siskiyou,-122.540597
2,41.589786,MULTIPOLYGON (((-121.44763489811575 41.9973336...,Modoc,-120.725183
3,40.698631,MULTIPOLYGON (((-124.06516973640174 41.4645760...,Humboldt,-123.873714
4,40.650636,MULTIPOLYGON (((-122.67701482828117 41.2803710...,Trinity,-123.112688


This is how the location data looks like.

In [3]:
# dropping the unnecessary column
ca_counties = ca_counties_df.drop('the_geom', axis=1)

# renaming a coulumn
ca_counties.rename(columns={'Name': 'County'}, inplace=True)

ca_counties.head()

,Latitude,County,Longitude
0,41.743438,Del Norte,-123.896950
1,41.592777,Siskiyou,-122.540597
2,41.589786,Modoc,-120.725183
3,40.698631,Humboldt,-123.873714
4,40.650636,Trinity,-123.112688


#### 2. Population Data

Let's download population data from above mentioned source.

In [4]:
! wget -q -O 'ca_counties_population.csv' https://www.dropbox.com/s/rdi7kebqo9dxq5r/ca_population.csv?dl=1

In [5]:
population_df = pd.read_csv('ca_counties_population.csv')
population_df.head()

,County,Population
0,".Alameda County, California",1671329
1,".Alpine County, California",1129
2,".Amador County, California",39752
3,".Butte County, California",219186
4,".Calaveras County, California",45905


The values of the 'County' column contain leading dots and have "County, California" added to them. We need to get rid of this for ease of analysis.

In [6]:
population_df['County'] = population_df['County'].apply(lambda x: x.replace(" County, California", ""))
population_df['County'] = population_df['County'].apply(lambda x: x.replace(".", ""))

In [7]:
population_df.dtypes

County        object
Population     int64
dtype: object

The columns of `population_df` has values with 'object' data type and integer data type.

The dataframe that I will work with in its final form-

In [8]:
population_df.head()

,County,Population
0,Alameda,1671329
1,Alpine,1129
2,Amador,39752
3,Butte,219186
4,Calaveras,45905


#### 3. Real GDP Data

In [9]:
! wget -q -O 'ca_counties_gdp.csv' https://www.dropbox.com/s/hlbrcmoksxshc6k/ca_real_gdp.csv?dl=1

Formatted data is downloaded from above mentioned source. 

In [10]:
gdp_df = pd.read_csv('ca_counties_gdp.csv')
gdp_df.rename(columns={'county': 'County', 'real_gdp': 'Real_GDP'}, inplace=True)
gdp_df.head()

,County,Real_GDP
0,Alameda,130701879
1,Alpine,214895
2,Amador,1651760
3,Butte,9219915
4,Calaveras,1440250


In [11]:
print(gdp_df.shape)

(58, 2)


#### Forming a unified dataframe with Location Data, GDP Data and Population Data

In [12]:
ca_counties_gdp_pop = population_df.merge(gdp_df, on='County')
print(ca_counties_gdp_pop.shape)
ca_counties_gdp_pop.head()

(58, 3)


,County,Population,Real_GDP
0,Alameda,1671329,130701879
1,Alpine,1129,214895
2,Amador,39752,1651760
3,Butte,219186,9219915
4,Calaveras,45905,1440250


In [13]:
ca_counties_df = ca_counties_gdp_pop.merge(ca_counties, on='County')
ca_counties_df.head()

,County,Population,Real_GDP,Latitude,Longitude
0,Alameda,1671329,130701879,37.644874,-121.882289
1,Alpine,1129,214895,38.597971,-119.820211
2,Amador,39752,1651760,38.446117,-120.651572
3,Butte,219186,9219915,39.667011,-121.600718
4,Calaveras,45905,1440250,38.205015,-120.553757


In [14]:
ca_counties_df.shape

(58, 5)

#### 5. Foursquare Data

Client ID and Client Secret has been stored in two variables `client_id` and `client_secret`. The code cell has been deleted for obvious security reasons.

In [16]:
CLIENT_ID = client_id
CLIENT_SECRET = client_secret
VERSION = '20200101'

Now, I shall call the Foursquare API to provide an outlook of the data.

In [17]:
# choosing the Los Angeles county of California and showing its venues

ca_counties.loc[52, 'County']

'Los Angeles'

In [18]:
# retrieving information about the county from dataframe

county_name = ca_counties.loc[52, 'County']
county_lat = ca_counties.loc[52, 'Latitude']
county_long = ca_counties.loc[52, 'Longitude']

print(f'The {county_name} county\'s latitude is {county_lat} and longitude is {county_long}')

The Los Angeles county's latitude is 34.3615935047 and longitude is -118.21698181100001


Now let's retrieve the list of venues which are in the vicinity of the center of the county. A list of venues within 10 km will be retrieved.

In [19]:
# getting venues from Foursquare API

RADIUS = 10000
category_id = '4d4b7105d754a06374d81259' # category id for food as provided in Foursquare API documentation
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    county_lat,
    county_long,
    category_id,
    RADIUS,    
)

url

'https://api.foursquare.com/v2/venues/search?&client_id=LNBN1GEEE0442WVW4EBRG2AP2M5D2INS422JHZVKSZ0WOM0V&client_secret=EJ1DEZ5HCP3I05RNZ4GZYOUKWGSNYPP30UF4RPIIW3EH45RR&v=20200101&ll=34.3615935047,-118.21698181100001&categoryId=4d4b7105d754a06374d81259&radius=10000'

In [20]:
results = requests.get(url).json()

In [21]:
results

{'meta': {'code': 200, 'requestId': '5f11ccbd417c2434e4590849'},
 'response': {'confident': False,
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_foodcourt_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d120951735',
      'name': 'Food Court',
      'pluralName': 'Food Courts',
      'primary': True,
      'shortName': 'Food Court'}],
    'hasPerk': False,
    'id': '4fc2d02be4b021b6e05b532e',
    'location': {'cc': 'US',
     'city': 'La Crescenta',
     'country': 'United States',
     'distance': 13707,
     'formattedAddress': ['La Crescenta, CA 91214', 'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.240134,
       'lng': -118.24148}],
     'lat': 34.240134,
     'lng': -118.24148,
     'postalCode': '91214',
     'state': 'CA'},
    'name': "In N' Out Burger",
    'referralId': 'v-1595002533'},
   {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
   

In [22]:
venues = results['response']['venues']

In [23]:
venues
nearby_venues = pd.json_normalize(venues)
nearby_venues.head(3)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address
0,4fc2d02be4b021b6e05b532e,In N' Out Burger,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1595002533,False,34.240134,-118.241480,"[{'label': 'display', 'lat': 34.240134, 'lng':...",13707,91214,US,La Crescenta,CA,United States,"[La Crescenta, CA 91214, United States]",NaN
1,4ee011c20e01452f91ee0d76,Wences Bistro,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1595002533,False,34.467690,-118.196388,"[{'label': 'display', 'lat': 34.46768951416015...",11960,93510,US,Acton,CA,United States,"[3620 Smith Ave, Acton, CA 93510, Acton, CA 93...","3620 Smith Ave, Acton, CA 93510"
2,4f325f7819836c91c7d160db,Rio Cafe and Groceries,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1595002533,False,34.440215,-118.297116,"[{'label': 'display', 'lat': 34.440215, 'lng':...",11435,91390,US,Santa Clarita,CA,United States,"[9411 Soledad Canyon Rd, Santa Clarita, CA 913...",9411 Soledad Canyon Rd


In [24]:
nearby_venues.shape

(8, 16)

In [25]:
# defining function to extract category from the row 'categories' in the nearby_venues dataframe

def get_category(entry):
  return entry[0]['name']

In [26]:
# forming a clean dataframe

# keeping only relevant columns and truncating the rest
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# getting relevant information from categories
nearby_venues['categories'] = nearby_venues['categories'].apply(get_category)

# renaming columns
nearby_venues.rename(columns={
                        'categories': 'category',
                        'location.lat': 'latitude',
                        'location.lng': 'longitude'
}, inplace=True)

nearby_venues.head()

,name,category,latitude,longitude
0,In N' Out Burger,Food Court,34.240134,-118.241480
1,Wences Bistro,Café,34.467690,-118.196388
2,Rio Cafe and Groceries,Deli / Bodega,34.440215,-118.297116
3,Jack in the Box,Fast Food Restaurant,34.238508,-118.245921
4,Funnel Cake Stand,Dessert Shop,34.261764,-118.299381


This is the data for only one county- Los Angeles. A function will be designed to to fetch data for all counties for analysis.

Let's see a map of the listed eateries near the Los Angeles county.

In [32]:
# generating a map of California

map_la = folium.Map(location=[34.36159, -118.21698], zoom_start=12)

# add markers to map
for lat, lng, county in zip(nearby_venues['latitude'], nearby_venues['longitude'], nearby_venues['name']):
    label = '{}'.format(county)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#350091',
        fill=True,
        fill_color='#5e03fc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)
    
map_la